У нас появился запрос из отдела продаж и маркетинга. Как вы знаете «МегаФон»
предлагает обширный набор различных услуг своим абонентам. При этом разным
пользователям интересны разные услуги. Поэтому необходимо построить
алгоритм, который для каждой пары пользователь-услуга определит вероятность
подключения услуги.

### Данные

В качестве исходных данных вам будет доступна информация об отклике
абонентов на предложение подключения одной из услуг. Каждому пользователю
может быть сделано несколько предложений в разное время, каждое из которых он
может или принять, или отклонить.

Отдельным набором данных будет являться нормализованный
анонимизированный набор признаков, характеризующий профиль потребления
абонента. Эти данные привязаны к определенному времени, поскольку профиль
абонента может меняться с течением времени.

Данные train и test разбиты по периодам – на train доступно 4 месяцев, а на test
отложен последующий месяц.

Итого, в качестве входных данных будут представлены:

 - data_train.csv: id, vas_id, buy_time, target
 - features.csv.zip: id, <feature_list>

И тестовый набор:

- data_test.csv: id, vas_id, buy_time
    target - целевая переменная, где 1 означает подключение услуги, 0 - абонент
    не подключил услугу соответственно.
    
    buy_time - время покупки, представлено в формате timestamp, для работы с
    этим столбцом понадобится функция datetime.fromtimestamp из модуля
    datetime.
    
    id - идентификатор абонента
    
    vas_id - подключаемая услуга

Примечание: Размер файла features.csv в распакованном виде весит 20 гб, для
работы с ним можно воспользоваться pandas.read_csv, либо можно воспользоваться
библиотекой Dask.

### Метрика

Скоринг будет осуществляться функцией f1, невзвешенным образом, как например
делает функция sklearn.metrics.f1_score(…, average=’macro’)

### Формат представления результата

1. Работающая модель в формате pickle, которая принимает файл data_test.csv
из корневой папки и записывает в эту же папку файл answers_test.csv. В этом
файле должны находится 4 столбца: buy_time, id, vas_id и target. Target можно
записать как вероятность подключения услуги.
2. Код модели можно представить в виде jupyter-ноутбука.
 
3. Презентация в формате .pdf, в которой необходимо отразить:

 - Информация о модели, ее параметрах, особенностях и основных результатах.
 
 - Обоснование выбора модели и ее сравнение с альтернативами.
 
 - Принцип составления индивидуальных предложений для выбранных абонентов.

Рекомендуемое количество слайдов – 5 – 10.
Файл answers_test.csv с результатами работы модели, презентацию, ноутбуки и
резюме необходимо прикрепить ко второму уроку “курсовой проект”

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score, precision_recall_curve, classification_report

from sklearn.feature_selection import SelectFromModel, GenericUnivariateSelect, mutual_info_classif

In [ ]:
from until import undersample_df_by_target, run_grid_search, treshold_search, preprocess_data_train, preprocess_data_test, select_type_cols


In [ ]:
TRAIN_DATA = 'data/data_train.csv'
TEST_DATA = 'data/data_train.csv'
FEATURES_DATA = 'data/features.csv'
RANDOM_STATE = 9

In [ ]:
train_df = pd.read_csv(TRAIN_DATA)

In [ ]:
train_df.head(2)

##### Сперва достанем данные из временной метки, и посмотрим распределение целевого признака в разрезе остальных признаков.

In [ ]:
train_df = train_df.drop('Unnamed: 0', axis=1)

In [ ]:
train_df['buy_time'] = pd.to_datetime(train_df['buy_time'], unit='s') 

In [ ]:
train_df['monthday'] = train_df['buy_time'].dt.day
train_df['month'] = train_df['buy_time'].dt.month

- В процессе выяснил, что все звонки были в воскресенье в 21 час. По этому нет смысла добавлять день недели и время звонка. (А жаль, мне кажется было бы полезно)

In [ ]:
train_df.head(2)

In [ ]:
train_df = train_df.sort_values('buy_time')

In [ ]:
train_df['not_first_offer'] = train_df.duplicated('id').astype(int)

- Выделим пользователей, которым делали предложения больше 1 раза.

##### Целевая переменная

In [ ]:
plt.figure(figsize=(8, 5))

sns.countplot(x='target', data=train_df)

plt.title('Target variable distribution')
plt.show()

- Таргет имеет сильный дисбаланс

In [ ]:
plt.figure(figsize=(10, 8))

sns.countplot(x="vas_id", hue='target', data=train_df)
plt.title('vas_id grouped by target variable')
plt.legend(title='Target', loc='upper right')

plt.show()

- Видим, что на услугу "6" положительный отклик, относительно отрицательного, на много выше остальных. Также "4" услуга имеет больший спрос. 

In [ ]:
plt.figure(figsize=(10, 8))

sns.countplot(x="monthday", hue='target', data=train_df)
plt.title('monthday grouped by target variable')
plt.legend(title='Target', loc='upper right')

plt.show()

- Наблюдается сильный прекос отклика в положительную сторону в середине месяца, может это период зарплаты, когда люди более лояльны к новым покупкам. Ну и в целом видно, что в это время пик предложений, видимо маркетинг тоже пришел к выводу, что лучше звонить в эти дни. 

In [ ]:
plt.figure(figsize=(10, 8))

sns.countplot(x="not_first_offer", hue='target', data=train_df)
plt.title('not_first_offer grouped by target variable')
plt.legend(title='Target', loc='upper right')

plt.show()

- Интересно, что те, кому звонили не в первый раз, имеют явный перекос в сторону положительного отклика. 

In [ ]:
plt.figure(figsize=(10, 8))

sns.countplot(x="month", hue='target', data=train_df)
plt.title('month grouped by target variable')
plt.legend(title='Target', loc='upper right')

plt.show()

- Видно, что в Ноябре \ Декабре имеется явное увеличение положительных откликов. В декабре большое количество предложений, возможно предлагали новогоднюю акцию или старались закрыть план за год.

In [ ]:
sample = train_df.loc[(train_df['not_first_offer'] == 1) & (train_df['monthday'].isin([16, 17, 18, 19, 20]))]

In [ ]:
sample['target'].value_counts(normalize=True)

In [ ]:
plt.figure(figsize=(10, 8))

sns.countplot(x="vas_id", hue='target', data=sample)
plt.title('vas_id grouped by target variable')
plt.legend(title='Target', loc='upper right')

plt.show()

##### Из приведенных выше данных, можно сделать предположение: "Всегда нужно звонить повторно, звонить стоит в середине месяца". При этом не чаще раза в месяц, и предлагать разные услуги. Даже без дополнительных описательных характеристик пользователя, мы видим, что вероятность положительного отклика на любую услугу в этих рамках много выше. 

#### Посмотрим описательные характеристики

In [ ]:
features_df = dd.read_csv(FEATURES_DATA, sep='\t')

In [ ]:
features_df.head()

In [ ]:
features_df = features_df.drop('Unnamed: 0', axis=1)

- Чтобы смерджить описательные фичи, возьмем из них только те id, которые есть в трейн датасете.

In [ ]:
train_list_index = list(train_df['id'].unique())

In [ ]:
features_df = features_df.loc[features_df['id'].isin(train_list_index)].compute()

- Т.к. имеются дубликаты ИД в описательном датасете, подразумевается, что со временем предпочтения менялись берем ближайшую по времени информацию. Если данных не будет, будет nan который мы обработаем в пайплайне заполнив на какую-нибудь статистику.

In [ ]:
features_df['buy_time'] = pd.to_datetime(features_df['buy_time'], unit='s')

In [ ]:
features_df = features_df.sort_values(by="buy_time")

In [ ]:
train_data = pd.merge_asof(train_df, features_df, on='buy_time', by='id', direction='nearest')

- Проверим, совпадают ли значения

In [ ]:
import random

rid = random.choice(train_list_index)
print(rid)
features_df.loc[features_df['id'] == rid]


In [34]:
train_data.loc[train_data['id'] == rid]

,id,vas_id,buy_time,target,monthday,month,not_first_offer,0,1,2,...,243,244,245,246,247,248,249,250,251,252
663749,224359,2.0,2018-12-16 21:00:00,0.0,16,12,0,245.580029,-65.799112,257.059214,...,-710.373846,-376.770792,-25.996269,-37.630448,-241.747724,-4.832889,-0.694428,8.824067,-0.45614,0.0


In [35]:
train_data.drop(['id', 'buy_time', 'month'], axis=1, inplace=True)

In [36]:
print("ID уникален? ", train_data.index.is_unique)
print("Есть ли дубли в строках?", train_data.duplicated().sum())
print("Сколько процент признаков могут принимать null-значениями? %d%%" % float((train_data.isnull().sum() > 0).sum()/train_data.shape[1]*100))

ID уникален?  True
Есть ли дубли в строках? 732
Сколько процент признаков могут принимать null-значениями? 0%


In [37]:
train_data.drop_duplicates(inplace=True)

In [38]:
train_data['target'].value_counts()

0.0    770747
1.0     60174
Name: target, dtype: int64

- Посмотрим на корреляцию между целевой и остальными признаками. 

In [39]:
corr_matrix = train_data.corr()[['target']]

In [40]:
corr_matrix.loc[abs(corr_matrix['target']) > 0.3].shape[0] - 1

1

In [41]:
corr_matrix.loc[abs(corr_matrix['target']) > 0.2].shape[0] - 1

2

In [42]:
corr_matrix.loc[abs(corr_matrix['target']) > 0.1].shape[0] - 1

2

In [43]:
corr_matrix.loc[abs(corr_matrix['target']) > 0.05].shape[0] - 1

2

In [44]:
corr_matrix.loc[abs(corr_matrix['target']) > 0.01].shape[0] - 1

2

In [45]:
corr_matrix.loc[abs(corr_matrix['target']) > 0.001]

,target
vas_id,0.262972
target,1.000000
monthday,0.007250
not_first_offer,0.372296
0,0.001181
...,...
241,-0.004290
243,-0.001036
245,0.003425
247,-0.001163


 - Очень слабая корреляция. Либо связь не линейная т.к. имеются по сути категориальные признаки, либо просто признаки бесполезные. 

### Подготовим данные, разобьем на трейн\тест по времени.

In [46]:
train_df = pd.read_csv(TRAIN_DATA)

- Разделим на трейн\тест. Берем половину последнего месяца.

In [47]:
train_df['buy_time'] = pd.to_datetime(train_df['buy_time'], unit='s')

In [48]:
new_train_df = train_df.loc[~((train_df['buy_time'].dt.month == 12) & (train_df['buy_time'].dt.day > 10))]

In [49]:
valid_df = train_df.loc[((train_df['buy_time'].dt.month == 12) & (train_df['buy_time'].dt.day > 10))]

- Сделаем балансировку андерсемплингом так как данных много

In [50]:
X_train = undersample_df_by_target(new_train_df, 'target')

In [51]:
X_train, true_offers_ids = preprocess_data_train(X_train, FEATURES_DATA)

In [52]:
y_train = X_train['target']

In [53]:
X_train = X_train.drop('target', axis = 1)

In [54]:
y_train.value_counts()

1.0    45094
0.0    45094
Name: target, dtype: int64

In [55]:
valid_df = preprocess_data_test(valid_df, FEATURES_DATA, true_offers_ids)

C:\Users\user\AppData\Local\Temp/ipykernel_18520/2783419716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prep_data_df['buy_time'] = pd.to_datetime(prep_data_df['buy_time'], unit='s')


In [56]:
y_test = valid_df['target']

In [57]:
X_test = valid_df.drop('target', axis = 1)

In [58]:
y_test.value_counts()

0.0    178557
1.0     15083
Name: target, dtype: int64

- И так, для начала сделаем Бейзлайн, оценив, что мы можем получить.

In [60]:
rf = RandomForestClassifier()

In [61]:
step_imputer = SimpleImputer(strategy="mean")

In [62]:
baseline = Pipeline([
    ('imuter', step_imputer),
    ('model', rf)
])

In [63]:
baseline.fit(X_train, y_train)

Pipeline(steps=[('imuter', SimpleImputer()),
                ('model', RandomForestClassifier())])

In [64]:
preds = baseline.predict(X_test)

In [65]:
f1_score(y_test, preds, average='macro')

0.5620124324464298

In [66]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       1.00      0.66      0.79    178557
         1.0       0.20      0.99      0.33     15083

    accuracy                           0.69    193640
   macro avg       0.60      0.83      0.56    193640
weighted avg       0.94      0.69      0.76    193640



- Весьма не плохой результат для использования данных "как есть", F-Score=0.872, F1-macro 0.67

===========================================================================================================

#### Попробуем глянуть, что за фичи мы имеем.

- И так, соберем списки признаков

In [67]:
f_all, f_binary, f_categorical, f_numeric = select_type_cols(X_train)

- Создадим пайплайн для препроцессинга. В нем мы стандартизируем вещественные признаки, что бы попробовать использовать линейную модель, и и сделаем OHE категориальных признаков. Также добавим шаги по автозаполнению пропусков.  

In [68]:
f_prep_pipeline = make_pipeline(
    ColumnSelector(columns=f_all),
    FeatureUnion(transformer_list=[
        ("numeric_features", make_pipeline(
            ColumnSelector(f_numeric),
            SimpleImputer(strategy="mean"),
            StandardScaler()
        )),
        ("categorical_features", make_pipeline(
            ColumnSelector(f_categorical),
            SimpleImputer(strategy="most_frequent"),
            OneHotEncoder(handle_unknown='ignore')
        )),
        ("boolean_features", make_pipeline(
            ColumnSelector(f_binary),
        ))
    ])
)

- Попробуем наш бейзлайн на новых фичах. По сути для Леса изменилось только кодирование кат.фич.

In [69]:
rf_pipe = make_pipeline(
    f_prep_pipeline,
    RandomForestClassifier(random_state=RANDOM_STATE)
)

In [70]:
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['45', '137',
                                                          'not_first_offer',
                                                          '22', '41', '87',
                                                          '119', '248', '241',
                                                          '116', '214', '233',
                                                          '6', '55', '46',
                                                          '187', '91', '95',
                                                          '51', '115', '217',
                                                          '230', '105', '170',
                                                          '172', '179', '70',
                                                          '210', '216', '178', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(s...
                                                                                                           '175',
                                                                                                           '177',
                                                                                                           '95',
                                                                                                           '221',
                                                                                                           '252',
                                                                                                           '192',
                                                                                                           '32',
                                                                                                           '179',
                                                                                                           '70', ...])),
                                                                                  ('simpleimputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('onehotencoder',
                                                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                                                 ('boolean_features',
                                                                  Pipeline(steps=[('columnselector',
                                                                                   ColumnSelector(columns=['not_first_offer']))]))]))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=9))])

In [71]:
preds = rf_pipe.predict(X_test)

In [72]:
f1_score(y_test, preds, average='macro')

0.7472968293782207

In [73]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93    178557
         1.0       0.39      1.00      0.56     15083

    accuracy                           0.88    193640
   macro avg       0.70      0.93      0.75    193640
weighted avg       0.95      0.88      0.90    193640



- Не плохой прирост, +4% точности F1-macro, это хороший результат.

========================================================================================================================

- Теперь попробуем линейный алгоритм.

In [74]:
lg_pipe = make_pipeline(
    f_prep_pipeline,
    LogisticRegression(random_state=RANDOM_STATE)
)

In [75]:
lg_pipe.fit(X_train, y_train)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['45', '137',
                                                          'not_first_offer',
                                                          '22', '41', '87',
                                                          '119', '248', '241',
                                                          '116', '214', '233',
                                                          '6', '55', '46',
                                                          '187', '91', '95',
                                                          '51', '115', '217',
                                                          '230', '105', '170',
                                                          '172', '179', '70',
                                                          '210', '216', '178', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(s...
                                                                                                           '155',
                                                                                                           '163',
                                                                                                           '175',
                                                                                                           '177',
                                                                                                           '95',
                                                                                                           '221',
                                                                                                           '252',
                                                                                                           '192',
                                                                                                           '32',
                                                                                                           '179',
                                                                                                           '70', ...])),
                                                                                  ('simpleimputer',
                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                  ('onehotencoder',
                                                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                                                 ('boolean_features',
                                                                  Pipeline(steps=[('columnselector',
                                                                                   ColumnSelector(columns=['not_first_offer']))]))]))])),
                ('logisticregression', LogisticRegression(random_state=9))])

In [76]:
preds = lg_pipe.predict(X_test)

In [77]:
f1_score(y_test, preds, average='macro')

0.7485851896299329

In [78]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93    178557
         1.0       0.40      1.00      0.57     15083

    accuracy                           0.88    193640
   macro avg       0.70      0.93      0.75    193640
weighted avg       0.95      0.88      0.90    193640



- Результат очень близкий, при этом скорость на много выше.

========================================================================================================================

- Произведем отбор фичей. Попробуем использовать метод с Лассо регрессией и Взаимной информации. В качестве контрольной модели будем использовать Лог рег, т.к. она хорошо себя показала, и работает довольно быстро.

In [79]:
lg_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l2', random_state=RANDOM_STATE, solver='liblinear'), threshold=1e-5),
    LogisticRegression(random_state=RANDOM_STATE)
)

In [80]:
params = [
    {"selectfrommodel__max_features": [None,15, 30,80,120,200,250],
     "selectfrommodel__threshold": [-np.inf],
     "selectfrommodel__estimator__C" : [1, 0.5, 0.01, 0.1]
    }
    
]

run_grid_search(lg_fs_pipe, X_train, y_train, params, scoring='f1')

Best f1 score: 0.90

Best parameters set found on development set:

{'selectfrommodel__estimator__C': 1, 'selectfrommodel__max_features': 5, 'selectfrommodel__threshold': -inf}


- Удивительно, но даже при 5 фичах качество практически такое же, как и при полном наборе.

========================================================================================================================

In [81]:
lg_fs_pipe_kbest_selector = make_pipeline(
    f_prep_pipeline,
    GenericUnivariateSelect(score_func=mutual_info_classif, mode='k_best', param=100),
    LogisticRegression(random_state=RANDOM_STATE)
)


In [82]:
params = [
    {'genericunivariateselect__param' : [15, 30,50,70,80,100,120,150,200,250]},
    
]


Best f1 score: 0.90

Best parameters set found on development set:

{'genericunivariateselect__param': 70}


- Будем использовать: SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'), threshold=1e-5). Т.к. 


==================================================================================================

In [83]:
rf_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'), max_features = 29, threshold = -np.inf),
    RandomForestClassifier(random_state=RANDOM_STATE, )
)

In [84]:
params = [
    {'randomforestclassifier__max_features': ['sqrt', 'log2', 2, 5, 8, 10],
     'randomforestclassifier__n_estimators' : [50, 100, 200, 300],    
    }   
]

Best f1 score: 0.90

Best parameters set found on development set:

{'randomforestclassifier__max_features': 8, 'randomforestclassifier__n_estimators': 300}

In [85]:
rf_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'), max_features = 29, threshold = -np.inf),
    RandomForestClassifier(random_state=RANDOM_STATE, max_features = 8, n_estimators = 300)
)

In [86]:
rf_fs_pipe.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['45', '137',
                                                          'not_first_offer',
                                                          '22', '41', '87',
                                                          '119', '248', '241',
                                                          '116', '214', '233',
                                                          '6', '55', '46',
                                                          '187', '91', '95',
                                                          '51', '115', '217',
                                                          '230', '105', '170',
                                                          '172', '179', '70',
                                                          '210', '216', '178', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(s...
                                                                 ('boolean_features',
                                                                  Pipeline(steps=[('columnselector',
                                                                                   ColumnSelector(columns=['not_first_offer']))]))]))])),
                ('selectfrommodel',
                 SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                              random_state=9,
                                                              solver='liblinear'),
                                 max_features=29, threshold=-inf)),
                ('randomforestclassifier',
                 RandomForestClassifier(max_features=8, n_estimators=300,
                                        random_state=9))])

In [87]:
preds_train = rf_fs_pipe.predict(X_train)
f1_score(y_train, preds_train, average='macro')

0.8728464899801937

In [88]:
preds_test = rf_fs_pipe.predict(X_test)
f1_score(y_test, preds_test, average='macro')

0.7462046840563894

In [89]:
preds_proba_train = rf_fs_pipe.predict_proba(X_train)

In [90]:
preds_proba_test = rf_fs_pipe.predict_proba(X_test)

In [91]:
treshold_search(y_train, preds_proba_train)

Лучшая отсечка : 0.5789473684210527, Метрика F1_macro: 0.8728466235532677
              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87     45094
         1.0       0.87      0.88      0.87     45094

    accuracy                           0.87     90188
   macro avg       0.87      0.87      0.87     90188
weighted avg       0.87      0.87      0.87     90188



In [92]:
treshold_search(y_test, preds_proba_test)

Лучшая отсечка : 0.6842105263157894, Метрика F1_macro: 0.7485720857707521
              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93    178557
         1.0       0.40      1.00      0.57     15083

    accuracy                           0.88    193640
   macro avg       0.70      0.93      0.75    193640
weighted avg       0.95      0.88      0.90    193640



=======================================================================================================

In [93]:
rf_gu_pipe = make_pipeline(
    f_prep_pipeline,
    GenericUnivariateSelect(score_func=mutual_info_classif, mode='k_best', param=50),
    RandomForestClassifier(random_state=RANDOM_STATE, n_estimators = 300)
)

In [94]:
rf_gu_pipe.fit(X_train, y_train)

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and bin

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and bin

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and bin

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and bin

C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)
C:\Users\user\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\cluster\_supervised.py:64: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and bin

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['45', '137',
                                                          'not_first_offer',
                                                          '22', '41', '87',
                                                          '119', '248', '241',
                                                          '116', '214', '233',
                                                          '6', '55', '46',
                                                          '187', '91', '95',
                                                          '51', '115', '217',
                                                          '230', '105', '170',
                                                          '172', '179', '70',
                                                          '210', '216', '178', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(s...
                                                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                                                 ('boolean_features',
                                                                  Pipeline(steps=[('columnselector',
                                                                                   ColumnSelector(columns=['not_first_offer']))]))]))])),
                ('genericunivariateselect',
                 GenericUnivariateSelect(mode='k_best', param=50,
                                         score_func=<function mutual_info_classif at 0x000002A9B4E8E4C0>)),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=300, random_state=9))])

In [95]:
preds_train = rf_gu_pipe.predict(X_train)
f1_score(y_train, preds_train, average='macro')

0.9965516466514099

In [96]:
preds_test = rf_gu_pipe.predict(X_test)
f1_score(y_test, preds_test, average='macro')

0.7466765882626827

In [97]:
preds_proba_train = rf_gu_pipe.predict_proba(X_train)

In [98]:
treshold_search(y_train, preds_proba_train)

Лучшая отсечка : 0.5263157894736842, Метрика F1_macro: 0.9965516473127713
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     45094
         1.0       1.00      1.00      1.00     45094

    accuracy                           1.00     90188
   macro avg       1.00      1.00      1.00     90188
weighted avg       1.00      1.00      1.00     90188



In [99]:
preds_proba_test = rf_gu_pipe.predict_proba(X_test)

In [100]:
treshold_search(y_test, preds_proba_test)

Лучшая отсечка : 0.6842105263157894, Метрика F1_macro: 0.7473460860822073
              precision    recall  f1-score   support

         0.0       1.00      0.88      0.93    178557
         1.0       0.40      0.97      0.56     15083

    accuracy                           0.88    193640
   macro avg       0.70      0.92      0.75    193640
weighted avg       0.95      0.88      0.90    193640



================================================================================================================

In [101]:
xgb_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'), max_features = 29, threshold = -np.inf),
    XGBClassifier(random_state=RANDOM_STATE)
)

In [102]:
params = [
    {'xgbclassifier__max_depth': [1,2,3,4,5],
     'xgbclassifier__n_estimators' : [200, 300, 400],     
    }    
]

Best parameters set found on development set:

{'xgbclassifier__max_depth': 2, 'xgbclassifier__n_estimators': 200}

In [103]:
xgb_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'), max_features = 29, threshold = -np.inf),
    XGBClassifier(random_state=RANDOM_STATE, max_depth = 2, n_estimators = 200)
)

In [104]:
xgb_fs_pipe.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:13:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['45', '137',
                                                          'not_first_offer',
                                                          '22', '41', '87',
                                                          '119', '248', '241',
                                                          '116', '214', '233',
                                                          '6', '55', '46',
                                                          '187', '91', '95',
                                                          '51', '115', '217',
                                                          '230', '105', '170',
                                                          '172', '179', '70',
                                                          '210', '216', '178', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(s...
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=2, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=200,
                               n_jobs=16, num_parallel_tree=1, predictor='auto',
                               random_state=9, reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=1, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbosity=None))])

In [105]:
preds_train = xgb_fs_pipe.predict(X_train)
f1_score(y_train, preds_train, average='macro')

0.8725359179835368

In [106]:
preds_test = xgb_fs_pipe.predict(X_test)
f1_score(y_test, preds_test, average='macro')

0.746436546554974

In [107]:
preds_proba_train = xgb_fs_pipe.predict_proba(X_train)

In [108]:
treshold_search(y_train, preds_proba_train)

Лучшая отсечка : 0.5263157894736842, Метрика F1_macro: 0.8725803624629289
              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87     45094
         1.0       0.87      0.88      0.87     45094

    accuracy                           0.87     90188
   macro avg       0.87      0.87      0.87     90188
weighted avg       0.87      0.87      0.87     90188



In [109]:
preds_proba_test = xgb_fs_pipe.predict_proba(X_test)

In [110]:
treshold_search(y_test, preds_proba_test)

Лучшая отсечка : 0.5789473684210527, Метрика F1_macro: 0.7487434903391735
              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93    178557
         1.0       0.40      1.00      0.57     15083

    accuracy                           0.88    193640
   macro avg       0.70      0.93      0.75    193640
weighted avg       0.95      0.88      0.90    193640



=================================================================================

In [137]:
lg_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'), max_features = 29),
    LogisticRegression(random_state=RANDOM_STATE)
)

In [138]:
lg_fs_pipe.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['45', '137',
                                                          'not_first_offer',
                                                          '22', '41', '87',
                                                          '119', '248', '241',
                                                          '116', '214', '233',
                                                          '6', '55', '46',
                                                          '187', '91', '95',
                                                          '51', '115', '217',
                                                          '230', '105', '170',
                                                          '172', '179', '70',
                                                          '210', '216', '178', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(s...
                                                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                                                 ('boolean_features',
                                                                  Pipeline(steps=[('columnselector',
                                                                                   ColumnSelector(columns=['not_first_offer']))]))]))])),
                ('selectfrommodel',
                 SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                              random_state=9,
                                                              solver='liblinear'),
                                 max_features=29)),
                ('logisticregression',
                 LogisticRegression(C=0.1, random_state=9,
                                    solver='liblinear'))])

In [139]:
preds_train = lg_fs_pipe.predict(X_train)
f1_score(y_train, preds_train, average='macro')

0.8724252998477848

In [140]:
preds_test = lg_fs_pipe.predict(X_test)
f1_score(y_test, preds_test, average='macro')

0.7488437802436829

In [115]:
preds_proba_train = lg_fs_pipe.predict_proba(X_train)

In [116]:
treshold_search(y_train, preds_proba_train)

Лучшая отсечка : 0.5, Метрика F1_macro: 0.8724252998477848
              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87     45094
         1.0       0.87      0.88      0.87     45094

    accuracy                           0.87     90188
   macro avg       0.87      0.87      0.87     90188
weighted avg       0.87      0.87      0.87     90188



In [117]:
preds_proba_test = lg_fs_pipe.predict_proba(X_test)

In [118]:
treshold_search(y_test, preds_proba_test)

Лучшая отсечка : 0.631578947368421, Метрика F1_macro: 0.7488556059932425
              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93    178557
         1.0       0.40      1.00      0.57     15083

    accuracy                           0.88    193640
   macro avg       0.70      0.93      0.75    193640
weighted avg       0.95      0.88      0.90    193640



In [119]:
lgbm_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'),max_features = 29, threshold=1e-5),
    LGBMClassifier(random_state=RANDOM_STATE)
)

In [120]:
params = [
    {
     'lgbmclassifier__n_estimators' : [100, 200, 300]     
    } 
]

In [121]:
lgbm_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l1', random_state=RANDOM_STATE, solver='liblinear'),max_features = 29, threshold=1e-5),
    LGBMClassifier(random_state=RANDOM_STATE, n_estimators = 200)
)

In [122]:
lgbm_fs_pipe.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['45', '137',
                                                          'not_first_offer',
                                                          '22', '41', '87',
                                                          '119', '248', '241',
                                                          '116', '214', '233',
                                                          '6', '55', '46',
                                                          '187', '91', '95',
                                                          '51', '115', '217',
                                                          '230', '105', '170',
                                                          '172', '179', '70',
                                                          '210', '216', '178', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(s...
                                                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                                                 ('boolean_features',
                                                                  Pipeline(steps=[('columnselector',
                                                                                   ColumnSelector(columns=['not_first_offer']))]))]))])),
                ('selectfrommodel',
                 SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                              random_state=9,
                                                              solver='liblinear'),
                                 max_features=29, threshold=1e-05)),
                ('lgbmclassifier',
                 LGBMClassifier(n_estimators=200, random_state=9))])

In [123]:
preds_train = lgbm_fs_pipe.predict(X_train)
f1_score(y_train, preds_train, average='macro')

0.8726023142003296

In [124]:
preds_test = lgbm_fs_pipe.predict(X_test)
f1_score(y_test, preds_test, average='macro')

0.746413771369073

In [125]:
preds_proba_train = lgbm_fs_pipe.predict_proba(X_train)

In [126]:
treshold_search(y_train, preds_proba_train)

Лучшая отсечка : 0.5789473684210527, Метрика F1_macro: 0.8726245368866256
              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87     45094
         1.0       0.87      0.88      0.87     45094

    accuracy                           0.87     90188
   macro avg       0.87      0.87      0.87     90188
weighted avg       0.87      0.87      0.87     90188



In [127]:
preds_proba_test = lgbm_fs_pipe.predict_proba(X_test)

In [128]:
treshold_search(y_test, preds_proba_test)

Лучшая отсечка : 0.631578947368421, Метрика F1_macro: 0.7487116724895408
              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93    178557
         1.0       0.40      1.00      0.57     15083

    accuracy                           0.88    193640
   macro avg       0.70      0.93      0.75    193640
weighted avg       0.95      0.88      0.90    193640



=================================================================================================

In [129]:
lg_fs_pipe = make_pipeline(
    f_prep_pipeline,
    SelectFromModel(LogisticRegression(penalty='l2', random_state=RANDOM_STATE, solver='liblinear', C = 0.1), max_features = 15),
    LogisticRegression(random_state=RANDOM_STATE)
)

In [130]:
lg_fs_pipe.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['45', '137',
                                                          'not_first_offer',
                                                          '22', '41', '87',
                                                          '119', '248', '241',
                                                          '116', '214', '233',
                                                          '6', '55', '46',
                                                          '187', '91', '95',
                                                          '51', '115', '217',
                                                          '230', '105', '170',
                                                          '172', '179', '70',
                                                          '210', '216', '178', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
                                                                  Pipeline(s...
                                                                                  ('onehotencoder',
                                                                                   OneHotEncoder(handle_unknown='ignore'))])),
                                                                 ('boolean_features',
                                                                  Pipeline(steps=[('columnselector',
                                                                                   ColumnSelector(columns=['not_first_offer']))]))]))])),
                ('selectfrommodel',
                 SelectFromModel(estimator=LogisticRegression(C=0.1,
                                                              random_state=9,
                                                              solver='liblinear'),
                                 max_features=15)),
                ('logisticregression', LogisticRegression(random_state=9))])

In [131]:
preds_train = lg_fs_pipe.predict(X_train)
f1_score(y_train, preds_train, average='macro')

0.8724252998477848

In [132]:
preds_test = lg_fs_pipe.predict(X_test)
f1_score(y_test, preds_test, average='macro')

0.7488437802436829

In [133]:
preds_proba_train = lg_fs_pipe.predict_proba(X_train)

In [134]:
treshold_search(y_train, preds_proba_train)

Лучшая отсечка : 0.5, Метрика F1_macro: 0.8724252998477848
              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87     45094
         1.0       0.87      0.88      0.87     45094

    accuracy                           0.87     90188
   macro avg       0.87      0.87      0.87     90188
weighted avg       0.87      0.87      0.87     90188



In [135]:
preds_proba_test = lg_fs_pipe.predict_proba(X_test)

In [136]:
treshold_search(y_test, preds_proba_test)

Лучшая отсечка : 0.5, Метрика F1_macro: 0.7488437802436829
              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93    178557
         1.0       0.40      1.00      0.57     15083

    accuracy                           0.88    193640
   macro avg       0.70      0.93      0.75    193640
weighted avg       0.95      0.88      0.90    193640



### Итог

Принял решение взять простую линейную модель, с отбором признаков методом l1 регуляризации.


Если сравнивать по weighted avg метрике (т.к. на тесте мы на трейне делали балансировку, а на тесте нет), то модель не склонна переобучиваться. При этом качество более сложных модели практически одинаково, а иногда и немного хуже, в то время, как ЛогРег работает быстрее прочих.

Лучшая отсечка по вероятности : 0.632, Метрика F1_macro: 0.749 на тестовой выборке.